## sfc 网络数据生成

**前提：**
 1. 网络结构最大5\*5,否则网络过大会造成数据量爆炸。
 2. Qos属性数量最大为4。理由同上。
 3. 想象中，请求从前向后依次经过每一类VNF，因为这里是加性的权重，经过顺序其实没有关系。
 4. Qos属性的取值范围确定。反正进行归一化之后，范围没有任何影响。


**可变参数：**
 1. 网络中存在的VNF。在5\*5的网络结构矩阵中，存在的VNF取值为1，否则为0。
 2. VNF的Qos取值。由于是4维属性，那么就是5\*5\*4的矩阵。
 3. 前后向属性的个数。前向属性，Qos取值乘上-1。
 4. 每一个属性的权重。4维属性，那么将每一个属性的权重扩张为5\*5的矩阵，组成5\*5\*4的矩阵。
 5. Qos请求。4维属性，与上面相同，扩张为5\*5\*4的矩阵。

## 样本形式

**样本属性：**

4层Qos属性取值，4层Qos属性权重，4层Qos请求，总共12层，5\*5\*12？

**样本标签：**

5\*6的矩阵，第一行有6列，每一列代表一个分类，那么就是选1、2、3、4、5或者不选（也就是不满足）。


------------------------------------------------------------------------
---


## 模型代码

In [ ]:
# 模型生成函数，负责依据参数生成sfc模型。

class sfc_model:

    import numpy as np
    
    # 网格大小
    max_shape = [5,5]

    # Qos 的属性个数
    L = 4

    # 每一个属性的取值范围
    scope = [1, 100]
    
    
    vnf_count = None # 每一类VNF的个数
    vnf_qos = None # Qos矩阵
    mask = None # 所需的Qos属性
    req = None # Qos请求
    w_qos = None # Qos权重
    k_qos = None # 前后向Qos

    # 构造函数
    def __init__(self, max_shape = [5, 5], L = 4, scope = [1, 100]):
        self.max_shape = max_shape
        self.L = L
        self.scope = scope
    
    
    ######################################################
    '''
    第一大步，随机出所有需要的参数：

    第一步，确定每一类VNF的个数。
    第二步，随机出一个Qos请求，随机出每一个Qos属性的权重。
    第三步，确定出需要使用的Qos属性，不需要的属性对应的取值置为0。
    第四步，确定Qos属性的前向以及后向性。
    '''
    def __random_all(self):
        vnf_count = self.np.random.randint(self.max_shape[1] + 1, size = self.max_shape[0])
        while self.np.sum(vnf_count) == 0:
            vnf_count = self.np.random.randint(self.max_shape[1] + 1, size = self.max_shape[0])
        
        vnf_qos = self.np.random.rand(self.max_shape[0], self.max_shape[1], self.L) * (self.scope[1] - self.scope[0]) + self.scope[0]
        
        for i in range(len(vnf_count)):
            for j in range(self.max_shape[1], vnf_count[i], -1):
                vnf_qos[i,j-1,:] = 0
        
        mask = self.np.random.rand(self.L) < 0.3
        while self.np.sum(mask) == self.L:
            mask = self.np.random.rand(self.L) < 0.3
        
        vnf_qos[:,:,mask] = 0
        
        req = self.np.random.rand(self.L) * (self.scope[1] * self.max_shape[0] * 3 - self.scope[0]) + self.scope[0]
        req[mask] = 0
        
        w_qos = self.np.random.rand(self.L)
        w_qos[mask] = 0
        w_qos = w_qos / self.np.sum(w_qos)
        
        k_qos = self.np.ones(self.L, dtype = self.np.int)
        k_qos[self.np.random.rand(self.L) < 0.5] = -1
        
        for i in range(len(req)):
            if k_qos[i] == 1:
                req[i] = req[i] / (self.max_shape[0] * 5)
        
        self.vnf_count = vnf_count
        self.vnf_qos = vnf_qos
        self.mask = mask
        self.req = req
        self.w_qos = w_qos
        self.k_qos = k_qos
        
        
    ######################################################    
    '''
    第二大步，暴力求解最优解：

    '''
    # 每一个 VNF 的 utility
    U = None

    # 最优路径的 utility
    best_utility = None

    # 最优路径
    best_path = None

    # 供 __computerU 使用，计算一个 VNF 的 utility
    def __computerPerU(self, qos):
        utility = 0.0
        for i in range(len(qos)):
            score = 0.0
            if self.k_qos[i] == 1:
                score = self.w_qos[i] * (qos[i] - self.scope[0]) / (self.scope[1] - self.scope[0])
            else:
                score = self.w_qos[i] * (self.scope[1] - qos[i]) / (self.scope[1] - self.scope[0])
            utility = utility + score
        return utility
    
    # 计算每一个 VNF 的 utility , 存入表 U
    def __computerU(self):
        self.U = self.np.zeros((self.max_shape[0], self.max_shape[1]))
        for i in range(self.max_shape[0]):
            for j in range(self.max_shape[1]):
                self.U[i,j] = self.__computerPerU(self.vnf_qos[i,j,:])
            

    # dfs到所有的路径，计算路径是否比当前最优值更优
    def __dfsPath(self, path, idx):
        if idx == self.max_shape[0]: # 路径结束
            
            this_qos = self.np.zeros(self.L)
            utility = 0.0
            for i in range(len(path)):
                if path[i] == self.max_shape[1]:
                    continue
                this_qos = this_qos + self.vnf_qos[i,path[i],:]
                utility = utility + self.U[i,path[i]]
            
            flag = True
            for i in range(self.L):
                if self.k_qos[i] == 1:
                    if this_qos[i] < self.req[i]:
                        flag = False
                        break
                else:
                    if this_qos[i] > self.req[i]:
                        flag = False
                        break
            
            if flag is True and ((self.best_utility is None) or (utility > self.best_utility)):
                self.best_utility = utility
                self.best_path = path
                
            return
        
        if self.vnf_count[idx] == 0:
            if path is None:
                path = [self.max_shape[1],]
            else:
                path.append(self.max_shape[1])
            self.__dfsPath(path, idx + 1)
        
        else:
            for i in range(self.vnf_count[idx]):
                path_copy = None
                if path is None:
                    path_copy = [i,]
                else:
                    path_copy = path.copy()
                    path_copy.append(i)
                self.__dfsPath(path_copy, idx + 1)
    
    
    def next_sample(self):
        self.__random_all()
        self.__computerU()
        self.best_path = None
        self.best_utility = None
        self.__dfsPath(None, 0)
        Y = self.np.zeros((self.max_shape[0], self.max_shape[1] + 1))
        if self.best_path is None:
            Y[:,self.max_shape[1]] = 1
        else:
            for i in range(self.max_shape[0]):
                Y[i,self.best_path[i]] = 1
        w_matrix = self.np.ones((self.max_shape[0], self.max_shape[1], self.L), dtype=self.np.int) * self.w_qos.reshape(1,1,-1)
        w_matrix = w_matrix * self.k_qos.reshape(1,1,-1)
        req_matrix = self.np.ones((self.max_shape[0], self.max_shape[1], self.L), dtype=self.np.int) * self.req.reshape(1,1,-1)
        X = self.np.dstack((self.vnf_qos,w_matrix,req_matrix))
        
        return X, Y


model.next_sample()

In [ ]:
import numpy as np

T = 100 * 100 * 1

model = sfc_model(max_shape = [5, 5], L = 4, scope = [1, 100])

X_data = np.zeros((T, 5, 5, 12))
Y_data = np.zeros((T, 5, 6))

for i in range(T):
    X_data[i,:,:,:], Y_data[i,:,:] = model.next_sample()
    
np.save("data/X_train_new", X_data)
np.save("data/Y_train_new", Y_data)


------------------------------------------------------------------------
---


## 卷积神经网络的构建

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Dense, Activation, BatchNormalization, Dropout, Conv2D, Flatten, ZeroPadding2D, MaxPooling2D
from keras.models import Model
from keras.initializers import glorot_uniform

In [ ]:
def my_net(input_len = (5,5,12), classes = 6 ):
    
    X_input = Input(input_len)
    
    # padding to 6 * 6
    X = ZeroPadding2D((1, 1))(X_input)
    
    # first conv
    X = Conv2D(filters = 64, kernel_size = (3, 3), strides = (1,1), padding = 'same', name = 'conv1', kernel_initializer = glorot_uniform())(X)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    
    # second conv
    X = Conv2D(filters = 128, kernel_size = (3, 3), strides = (1,1), padding = 'same', name = 'conv2', kernel_initializer = glorot_uniform())(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    
    # Pooling
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(512, activation='relu', name='fc1', kernel_initializer = glorot_uniform())(X)
    X = Dropout(0.5)(X)
    X = Dense(classes, activation='softmax', name='fc2', kernel_initializer = glorot_uniform())(X)

    model = Model(inputs = X_input, outputs = X, name='SFC_Net')
    
    return model

In [ ]:
X_train = np.load("data/X_train_new.npy")
Y_train = np.load("data/Y_train_new.npy")

In [ ]:
Net = {}

for i in range(5):
    net = my_net(input_len = (5,5,12), classes = 6 )
    net.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    net.fit(X_train, Y_train[:,i,:], epochs = 20, batch_size = 32, verbose = 1)
    Net["net" + str(i)] = net

In [ ]:
X_test = np.load("data/X_test_new.npy")
Y_test = np.load("data/Y_test_new.npy")

In [ ]:
unlike = np.zeros(Y_test.shape[0])

for i in range(5):
    net = Net["net" + str(i)]
    pred = np.argmax(net.predict(X_test), axis=1)
    real = np.argmax(Y_test[:,i,:], axis=1)
    unlike[pred != real] = 1

print(1 - (np.sum(unlike)/len(unlike)))

In [ ]:
preds = Net["net1"].evaluate(X_test, Y_test[:,1,:], verbose=1)
print ("Test Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
for i in range(5):
    net = Net["net" + str(i)]
    net.save("model_advance/net" + str(i) + ".h5")